## 1. Khai báo thư viện

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

## 2. Đọc dữ liệu

In [ ]:
data = pd.ExcelFile("data/DATA-SET-VÒNG-1-CUỘC-THI-DATA-GOT-TALENT-2023.xlsx")
customer = pd.read_excel(data, "customer")
ticket = pd.read_excel(data, "ticket")
film = pd.read_excel(data, "film")

## 3. Mô tả dữ liệu

In [ ]:
# Kích thước dữ liệu của bảng customer
customer.shape

In [ ]:
# Dữ liệu 10 dòng đầu của bảng customer
customer.head(10)

In [ ]:
# Dữ liệu 10 dòng cuối của bảng customer
customer.tail(10)

In [ ]:
# Kiểu dữ liệu của bảng customer
customer.info()

In [ ]:
# Kích thước dữ liệu của bảng ticket
ticket.shape

In [ ]:
# Dữ liệu 10 dòng đầu tiên của bảng ticket
ticket.head(10)

In [ ]:
# Dữ liệu 10 dòng cuối cùng của bảng ticket
ticket.tail(10)

In [ ]:
# Kiểu dữ liệu của bảng ticket
ticket.info()

In [ ]:
# Kích thước dữ liệu của bảng film
film.shape

In [ ]:
# Dữ liệu 10 dòng đầu tiên của bảng film
film.head(10)

In [ ]:
# Dữ liệu 10 dòng cuối cùng của bảng film
film.tail(10)

In [ ]:
# Kiểu dữ liệu của bảng film
film.info()

**Chú thích**

- Bảng **`customer`**:
    - Xuất hiện dòng có giá trị ở cột `customerid` có một dấu chấm ở cuối khác so với các dòng khác
    - Những dòng có giá trị ở cột `job` là "teenager" thì sẽ bị thiếu giá trị ở cột `industry`
- Bảng **`ticket`**:
    - Không phải mỗi vé trong bảng là một lần đặt
    - Các vé có cùng lần đặt sẽ có cùng giá trị ở cột `orderid`
    - Có 4 vé trong bảng bị thiếu dữ liệu (Khách hàng đặt vé trên website)
- Bảng **`film`**:

## Bước 4: Làm sạch và xử lý dữ liệu

In [ ]:
# chuẩn hóa lại tên cột theo chung một chuẩn
customer.columns = [re.sub(" ","_",c).lower().strip() for c in customer.columns]
ticket.columns = [re.sub(" ","_",c).lower().strip() for c in ticket.columns]
film.columns = [re.sub(" ","_",c).lower().strip() for c in film.columns]

# sửa lại "customerid" của một khách hàng bị thừa dấu chấm
customer["customerid"] = [re.sub(r"\.", "", c) for c in customer["customerid"]]
ticket["customerid"] = [re.sub(r"\.", "", c) for c in ticket["customerid"]]

In [ ]:
# biến đổi cột "dob" thành định dạng ngày và thêm cột "age" để lưu tuổi của mỗi khách hàng
base_date = datetime(1899, 12, 30, 0, 0, 0)
customer["dob"] = [(base_date + timedelta(days=date)) for date in customer["dob"]]
current_day = datetime.now()
dob_list = customer["dob"]
age_list = []
for day in dob_list:
    age = current_day.year - day.year
    age_list.append(age)
customer.insert(2,"age",age_list)


# tính tổng số lượng khách hàng theo độ tuổi
age_count = customer["age"].value_counts().sort_index().reset_index()
age_count

In [ ]:
# loại bỏ những khách hàng có tuổi âm
customer = customer.drop(customer[customer["age"]<=0].index)
age_count = customer["age"].value_counts().sort_index().reset_index()
customer["dob"] = pd.to_datetime(customer["dob"])
customer["dob"] = customer["dob"].dt.strftime('%Y-%m-%d')
age_count

In [ ]:
# vẽ biểu đồ thể hiện số lượng khách hàng theo độ tuổi
fig, ax = plt.subplots(figsize=(15,5))

ax = plt.bar(x = "age", height= "count", data=age_count)
plt.xticks(np.arange(0,61,2))
plt.yticks(np.arange(0,501,50))
plt.show()

In [ ]:
# tính tổng số lượng khách hàng theo giới tính
gender_count = customer["gender"].value_counts()
gender_count

In [ ]:
# tính tổng số lượng khách hàng theo công việc
job_count = customer["job"].value_counts().reset_index()
job_count

In [ ]:
# vẽ biểu đồ thể hiện tổng số lượng khách hàng theo công việc
fix , ax = plt.subplots(figsize=(10,5))

x = [x for x in range(len(job_count))]
colors = plt.get_cmap('Blues')(np.linspace(0.2, 0.7, len(x)))

ax.pie(x="count",labels="job",colors=colors , autopct="%1.1f%%", data=job_count)
plt.show()

In [ ]:
# tính tổng số lượng vé được đặt trên website
website_order_count = ticket["orderid"].isnull().sum()
website_order_count

In [ ]:
# tính tổng số lượng vé theo thời gian chiếu
time_count = ticket["time"].value_counts().sort_values(ascending=False).reset_index()
time_count.head(10)

In [ ]:
# thêm cột "day_nam" hiển thị thứ từ cột "date"
day_list = ticket["date"].tolist()
day_name_list = []
for day in day_list:
    day_name = day.strftime("%A")
    day_name_list.append(day_name)
ticket.insert(7,"day_name",day_name_list)
ticket["saledate"] = ticket["saledate"].dt.strftime('%Y-%m-%d')
ticket["date"] = ticket["date"].dt.strftime('%Y-%m-%d')
ticket

In [ ]:
# tính tổng số lượng và doanh thu theo ngày trong tháng
ticket_count_month = ticket["date"].value_counts().sort_index().reset_index()
ticket_price_sum_month = ticket.groupby("date").sum("ticket_price")["ticket_price"].sort_index().reset_index()
ticket_month = pd.merge(ticket_count_month, ticket_price_sum_month, how="outer", on="date")
ticket_month

In [ ]:
# vẽ biểu đồ thể hiện tổng số lượng và doanh thu theo ngày trong tháng
fig, ax = plt.subplots(figsize=(10,5))

color1 = "orangered"
color2 = "royalblue"

ax.plot("date", "ticket_price", data=ticket_month, color=color1)
ax.set_ylabel('Doanh thu theo ngày trong tháng', color=color1)
ax.tick_params(axis='y', labelcolor=color1)
ax2 = ax.twinx()
ax2.plot("date", "count", data=ticket_month,color=color2)
ax2.set_ylabel('Tổng số lượng vé bán được theo ngày trong tháng', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)
ax.set_xticks(range(len(ticket_month["date"])))
ax.set_xticklabels(ticket_month["date"], rotation=90)
plt.show()

In [ ]:
# tính tổng số lượng và doanh thu theo ngày trong tuần
order = ["Monday", "Tuesday", "Wednesday","Thursday" , "Friday", "Saturday", "Sunday"]
ticket_count_week = ticket["day_name"].value_counts().reindex(order).reset_index()
ticket_price_sum_week = ticket.groupby("day_name").sum("ticket_price")["ticket_price"].reindex(order).reset_index()
ticket_week = pd.merge(ticket_count_week, ticket_price_sum_week, how= "outer", on = "day_name")
ticket_week

In [ ]:
# vẽ biểu đồ thể hiện tổng số lượng và doanh thu theo ngày trong tuần
fix, ax = plt.subplots(figsize=(10,5))

ax.bar(x = "day_name", height= "count", width=0.4, data=ticket_week, color=color1)
ax.set_ylabel('Doanh thu theo ngày trong tuần', color=color1)
ax.tick_params(axis='y', labelcolor=color1)
ax2 = ax.twinx()
ax2.plot("day_name", "ticket_price", data=ticket_week, color=color2)
ax2.set_ylabel('Tổng số lượng vé bán được theo ngày trong tuần', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)

plt.show()

In [ ]:
# xuất dữ liệu ra từng file tương ứng
customer.to_excel("data/customer.xlsx",index=False)
ticket.to_excel("data/ticket.xlsx", index=False)
film.to_excel("data/film.xlsx", index=False)